In [3]:
import httpx
import lxml.html

# Target URL
target_url = "https://www.zillow.com/Chicago%20IL/"

# Headers copied from your request
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Cache-Control": "no-cache",
    "Pragma": "no-cache",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.6 Safari/605.1.15",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Host": "www.zillow.com"
}

# Create a session with httpx
# Ojo acá es .Client httpx
with httpx.Client(headers=headers, follow_redirects=True) as client:
    response = client.get(target_url)

# Print the response status code and part of the content
print(f"Status Code: {response.status_code}")
print(response.text[:500])  # Print first 500 characters of the HTML

#Have our response to text
html = response.text
#print(html)






Status Code: 200
<!DOCTYPE html><html lang="en"><head><script id="web-platform-datadog-rum-init">
    (function(h,o,u,n,d) {
      h=h[d]=h[d]||{q:[],onReady:function(c){h.q.push(c)}}
      d=o.createElement(u);d.async=1;d.src=n
      n=o.getElementsByTagName(u)[0];n.parentNode.insertBefore(d,n)
    })(window,document,'script','https://www.datadoghq-browser-agent.com/us1/v5/datadog-rum.js','DD_RUM')
    window.DD_RUM.onReady(function() {
        
      
        const shouldSetupDataDogRum = false;
        const 
